In [1]:
import requests
def get_tw_stock_kd_and_volume(stock_id):
    url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=&stockNo={stock_id}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    resp = requests.get(url, headers=headers)
    data = resp.json()
    if 'data' in data and data['data']:
        closes = [float(row[6].replace(',', '')) for row in data['data']]
        lows = [float(row[5].replace(',', '')) for row in data['data']]
        highs = [float(row[4].replace(',', '')) for row in data['data']]
        volumes = [int(row[1].replace(',', '')) for row in data['data']]
        # 計算KD指標
        period = 9
        k_values, d_values = [], []
        k, d = 50, 50
        for i in range(len(closes)):
            if i < period - 1:
                k_values.append(None)
                d_values.append(None)
                continue
            low_min = min(lows[i-period+1:i+1])
            high_max = max(highs[i-period+1:i+1])
            rsv = (closes[i] - low_min) / (high_max - low_min) * 100 if high_max != low_min else 0
            k = k * 2/3 + rsv * 1/3
            d = d * 2/3 + k * 1/3
            k_values.append(round(k, 2))
            d_values.append(round(d, 2))
        total_volume = sum(volumes)
        return {
            'K': k_values[-1],
            'D': d_values[-1],
            'total_volume': total_volume
        }
    return None

{'K': 66.67, 'D': 58.33, 'total_volume': 300882192}


In [2]:
tw100_stock_ids = [
    '2330', '2317', '2454', '2308', '2412', '2881', '2882', '2886', '2885', '2884',
    '1301', '1303', '2002', '2303', '2301', '2302', '2327', '2357', '2382', '2395',
    '2408', '2409', '2603', '2609', '2615', '2801', '2823', '2912', '3008', '3045',
    '3231', '3711', '4904', '4938', '5871', '5880', '6505', '8046', '8069', '8150',
    '8210', '8261', '9910', '9911', '9917', '9921', '9933', '9938', '9945', '1101',
    '1102', '1216', '1227', '1256', '1304', '1326', '1402', '1476', '1504', '1513',
    '1536', '1605', '1710', '1722', '1802', '2006', '2014', '2027', '2105', '2201',
    '2207', '2227', '2305', '2324', '2347', '2356', '2368', '2379', '2385', '2392',
    '2401', '2439', '2448', '2481', '2601', '2606', '2607', '2618', '2707', '2809',
    '2812', '2834', '2855', '2880', '2890', '2915', '3006', '3017', '3034', '3044',
    '3056', '3189', '3234', '3296', '3702', '3706', '4906', '4919', '4931', '4958'
 ]

In [3]:

import requests
import datetime
result = []
for stock_id in tw100_stock_ids:
    data = get_tw_stock_kd_and_volume(stock_id)
    if data and data['K'] is not None and data['K'] < 10:
        # 取得近一年(12個月)的成交量
        volumes = []
        for i in range(0):
            today = datetime.date.today()
            year = today.year
            month = today.month - i
            if month <= 0:
                year -= 1
                month += 12
            date_str = f"{year}{month:02d}01"
            url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_id}'
            headers = {'User-Agent': 'Mozilla/5.0'}
            resp = requests.get(url, headers=headers)
            stock_data = resp.json()
            if 'data' in stock_data and stock_data['data']:
                volumes += [int(row[1].replace(',', '')) for row in stock_data['data']]
        if volumes and volumes[-1] == max(volumes):
            result.append({'stock_id': stock_id, 'K': data['K'], 'total_volume': data['total_volume']})

if result:
    print(result)
else:
    result = '沒有相符股票'
    print(result)

NameError: name 'tw50_stock_ids' is not defined

In [ ]:
import requests
bot_token = '8071359988:AAGzBu4XILdJBhGJNQcgcLtAiNG8PQwC3Dk'
chat_id = '7030809792'
msg = f'result: {result}'
url = f'https://api.telegram.org/bot{bot_token}/sendMessage'
params = {
    'chat_id': chat_id,
    'text': msg
}
resp = requests.get(url, params=params)
print(resp.text)
print(resp.status_code)